In [228]:
import pandas as pd

In [229]:
def get_unit(un_list):
    """
    Функция формирования названия единиц измерения.
        Args:
            un_list(list): список слов, 
            начиная со второго в описании ингредиента
            
        Returns:
            unit(str): название единицы измерения
            в именительном падеже единственного числа
        """
    # Словарь замены падежей для единиц измерения ингредиентов
    subst={
            'туки' : 'тука',
            'ожек' : 'ожка',
            'ожки' : 'ожка',
            'штук' : 'штука',
            'чика' : 'чик',
            'уска' : 'усок',
            'иков' : 'ик',
            'кана' : 'кан',
            'анки' : 'анка',
            'сков' : 'сок',
            'ебля' : 'ебель',
            'овки' : 'овка',
            'блей' : 'бель',
            'чков' : 'чок',
            'анов' : 'ан',
            'овок' : 'овка',
            'анок' : 'анка',
            'учка' : 'учок',
            'овых' : 'овая',
            'овые' : 'овая',
            'йных' : 'йная',
            'йные' : 'йная',
            'дэша' : 'дэш',
            'очки' : 'очка',
            'очек' : 'очка'
             }
    ul=[]
    for word in un_list:
        
        if len(word)>3:
            if word[-4:] in subst.keys():
                subst_key = word[-4:]
                word = word[:-4]+subst[subst_key]
            
        ul.append(word)
    unit = (' ').join(ul)
    return unit

In [230]:
# Считываем исходную таблицу рецептов из файла.
connections = pd.read_csv('all_recipes.csv')
connections.drop_duplicates(inplace=True)
connections.shape

(84144, 11)

In [231]:
# Формируем таблицу ingredient
ingredient = pd.DataFrame(columns=['id_ingr', 'name_ingr'])
ingredient['name_ingr'] = connections.ingr_name
ingredient.drop_duplicates(inplace=True)
ingredient.reset_index(inplace=True)
ingredient['id_ingr']=ingredient.index
ingredient.drop('index', axis=1, inplace=True)

In [232]:
# Заменяем ingr_name в таблице connections на id_ingr из таблицы ingredients
mapper = dict(zip(ingredient.name_ingr, ingredient.id_ingr))
connections['id_ingr'] = connections.ingr_name.apply(lambda x: mapper[x])
connections.drop(['ingr_name'], axis=1, inplace=True)
# Формируем столбец id_con в таблице connections из индекса, это будет PK всей db
connections['id_con'] = connections.index

In [233]:
# Разбиваем описание ингредиента на список слов
connections['list'] = connections.ingr_qty_unit.str.split(' ')
# Формируем столбец begin из первых слов описания ингредиента
connections['begin'] = connections.list.apply(lambda x: x[0])
# Формируем столбец end из списка остальных слов описания ингредиента
connections['end'] = connections.list.apply(lambda x: x[1:])
# Заменяем символы дробей в столбце begin из первых слов описания ингредиента
connections.begin = connections.begin.str.replace('½','0.5').replace('¼','0.25').replace(
    '⅓','0.33').replace('⅔','0.67').replace('¾','0.75').replace(',','.', regex=True)
# Формируем признак isalpha - первые слов описания ингредиента состоят из букв
connections['isalpha'] = connections.begin.str.isalpha()
# Формируем признак unit - названия единиц измерения
connections['unit'] = connections['end'].apply(get_unit)
# Формируем признак qty - количество ингредиента
connections['ingr_qty'] = connections.apply(lambda row : float(row.begin) if row.isalpha==False else 0, axis = 1)
# Уточняем признак unit: для текстовых названий единиц измерения используем всю исходную строку
connections['unit'] = connections.apply(lambda row: row.ingr_qty_unit if row.ingr_qty==0 else row.unit, axis = 1)
# Удаление временных признаков
connections.drop(['list', 'begin', 'end', 'isalpha'], axis=1, inplace=True)

In [234]:
# Создаем таблицу unit
unit = pd.DataFrame(connections.unit.unique(), columns=['name_unit'])
unit['id_unit'] = unit.index

In [235]:
# Заменяем названия ингредиентов в таблице connections на их id
mapper = dict(zip(unit.name_unit,unit.id_unit))
connections.unit = connections.unit.apply(lambda x: mapper[x])
connections.rename(columns = {'unit':'id_unit'}, inplace = True )

In [236]:
# Создаем таблицу recipe
recipe = connections.copy(deep=True)
recipe = recipe.groupby(by='name_recipe').first().reset_index()
# Формируем столбец id_recipe в таблице recipe из индекса
recipe['id_recipe'] = recipe.index
# Заменяем name_recipe в таблице connections на id_recipe из таблицы recipe
mapper = dict(zip(recipe.name_recipe, recipe.id_recipe))
connections['id_recipe'] = connections.name_recipe.apply(lambda x: mapper[x])
recipe['vegeterian'] = False
recipe.drop(['ingr_qty_unit', 'id_unit', 'ingr_qty', 'id_con', 'id_ingr'], axis=1, inplace=True)
recipe.rename(columns = {'meal_mode':'meal_type'}, inplace = True )

In [237]:
# После создания таблицы recipes удаляем лишние признаки из таблицы set_ingredients
connections.drop(['name_recipe', 'portions', 'calories', 'protein', 
                  'fat', 'carbo', 'text_recipe', 'cuisine', 'meal_mode'], axis=1, inplace=True)

In [238]:
# Запись в файлы
connections.to_csv('./connections.csv', index=False)
recipe.to_csv('./recipe.csv', index=False)
unit.to_csv('./unit.csv', index=False)
ingredient.to_csv('./ingredient.csv', index=False)

### Визуализация таблиц

In [239]:
connections.sample(5)

,ingr_qty_unit,id_ingr,id_con,id_unit,ingr_qty,id_recipe
15531,250 мл,204,15531,12,250.0,986
84033,1 штука,84,84033,1,1.0,5657
14389,2 зубчика,25,14389,10,2.0,3120
52283,2 чайные ложки,396,52283,6,2.0,5023
14870,140 мл,44,14870,12,140.0,6441


In [240]:
recipe.sample(5)

,name_recipe,cuisine,meal_type,portions,calories,protein,fat,carbo,text_recipe,id_recipe,vegeterian
2601,Китайские весенние роллы с грибами и рисом,Китайская кухня,Основные блюда,4.0,336.0,12.0,18.0,31.0,"Обжарить на оливковом масле чеснок, добавить м...",2601,False
4288,Морской жюльен из креветок и мидий,Французская кухня,Закуски,2.0,605.0,45.0,41.0,14.0,Перемешать на горячей сковороде сливочное масл...,4288,False
3961,Малиновые пряники,Русская кухня,Выпечка и десерты,6.0,475.0,8.0,19.0,70.0,"Сушеную малину высыпать в блендер и, включив е...",3961,False
1084,Глинтвейн на темном пиве,Европейская кухня,Напитки,4.0,141.0,2.0,0.0,23.0,Залить в сотейник примерно полторы бутылки тем...,1084,False
3728,Ленивые голубцы с томатным соусом,Русская кухня,Основные блюда,6.0,615.0,21.0,50.0,20.0,"Нарезать капусту тонкой соломкой, лук и морков...",3728,False


In [241]:
unit

,name_unit,id_unit
0,г,0
1,штука,1
2,пучок,2
3,кг,3
4,банка,4
5,столовая ложка,5
6,чайная ложка,6
7,по вкусу,7
8,на кончике ножа,8
9,головка,9


In [242]:
ingredient

,id_ingr,name_ingr
0,0,Майонез
1,1,Куриное яйцо
2,2,Зеленый лук
3,3,Куриное филе
4,4,Консервированные шампиньоны
...,...,...
2597,2597,Хурма королек
2598,2598,Икра креветки
2599,2599,Креветочные панцири
2600,2600,Говяжий край
